# Environment setup

## Google Drive mount
I'm using Google Colaboratory as my default platform, therefore I need to set up my environment to integrate it with Google Drive. You can skip this bit if you're working locally.

1. Mount Google Drive on the runtime to be able to read and write files. This will ask you to log in to your Google Account and provide an authorization code.
2. Create a symbolic link to a working directory 
3. Change the directory to the one where I cloned my repository.


In [1]:
# mount Google Drive on the runtime
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [2]:
# create a symbolic link to a working directory
!ln -s /content/gdrive/My\ Drive/Colab\ Notebooks/datacourage_wine /mydrive

# navigate to the working directory
%cd /mydrive

ln: failed to create symbolic link '/mydrive/datacourage_wine': File exists
/content/gdrive/My Drive/Colab Notebooks/datacourage_wine


## Libraries & functions
Let's now import the necessary libraries and function we're gonna use in this notebook.

- `tqdm.notebook` - loop progress bar for notebooks
- `timeit` - cell runtime check
- `numpy` - linear algebra
- `pandas` - data manipulation & analysis


In [3]:
import tqdm.notebook as tq
import timeit
import numpy as np
import pandas as pd
from scipy import stats
import re
import plotly
import plotly.graph_objects as go

# Data exploration

# Red wine dataset

# Load data
Load data using `pd.read_csv` function. The file is not really comma-separated, so we need to change the separator object to semicolon using `sep` parameter.

In [4]:
df_red = pd.read_csv('winequality-red.csv', sep=';')
df_red

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5


## Rename columns
For a better readibility let's rename the columns and replace spaces with an underscore.

In [5]:
old_column_names = df_red.columns.values.tolist()
new_column_names = [re.sub('\s', '_', col_name) for col_name in old_column_names]

df_red.columns = new_column_names
df_red.head()

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


## Type of data
Continuous or discreet

# Missing values/errors
Let's check if we have any missing values in the dataset.

In [6]:
for col_name in df_red.columns.values.tolist():
  n = df_red[col_name].isnull().values.any()
  if n == False:
    print(f'OK! "{col_name}" column has no missing values.')
  else:
    print(f'Warning! "{col_name}" column has some missing values.')

OK! "fixed_acidity" column has no missing values.
OK! "volatile_acidity" column has no missing values.
OK! "citric_acid" column has no missing values.
OK! "residual_sugar" column has no missing values.
OK! "chlorides" column has no missing values.
OK! "free_sulfur_dioxide" column has no missing values.
OK! "total_sulfur_dioxide" column has no missing values.
OK! "density" column has no missing values.
OK! "pH" column has no missing values.
OK! "sulphates" column has no missing values.
OK! "alcohol" column has no missing values.
OK! "quality" column has no missing values.


## Duplicate content


In [7]:
# df_red.drop_duplicates(inplace=True)
# df_red

# Premature model

## Dataset split

In [8]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression

X = df_red.drop('quality', axis=1)
y = df_red['quality']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

## Model

In [9]:
clf = LogisticRegression(max_iter=10000)
clf.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=10000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

## Cross-validation score

In [10]:
score_cv = cross_val_score(clf, X_train, y_train, cv=5, scoring='accuracy').mean()
score_cv

0.5879916317991631

# Data exploration

## Statistical distribution

### Box plot
Let's investigate the statistical features of our dataset using a box plot.

In [11]:
data = [
        go.Box(
            y = df_red[col_name],
            notched=True, # notched appearance
            showlegend=True,
            name=col_name,
            boxpoints='outliers' # display outliers
            ) for col_name in df_red.columns
]

fig = go.Figure(
    data=data
)

fig.show()

## Remove outliers
We can see there's quite a lot of outliers which can skew our later predictions. Let's clear them.

In [12]:
# def get_iqr_values(df_in, col_name):
#     median = df_in[col_name].median()
#     q1 = df_in[col_name].quantile(0.25) # 25th percentile / 1st quartile
#     q3 = df_in[col_name].quantile(0.75) # 7th percentile / 3rd quartile
#     iqr = q3-q1 #Interquartile range
#     minimum  = q1-1.5*iqr # The minimum value or the |- marker in the box plot
#     maximum = q3+1.5*iqr # The maximum value or the -| marker in the box plot
#     return median, q1, q3, iqr, minimum, maximum

In [13]:
# def remove_outliers(df_in, col_name):
#     _, _, _, _, minimum, maximum = get_iqr_values(df_in, col_name)
#     df_out = df_in.loc[(df_in[col_name] > minimum) & (df_in[col_name] < maximum)]
#     return df_out

In [14]:
# for col_name in df_red.columns:
#   df_red = remove_outliers(df_red, col_name)
# df_red


In [16]:
df_red.corr()

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,quality
fixed_acidity,1.000000,-0.256131,0.671703,0.114777,0.093705,-0.153794,-0.113181,0.668047,-0.682978,0.183006,-0.061668,0.124052
volatile_acidity,-0.256131,1.000000,-0.552496,0.001918,0.061298,-0.010504,0.076470,0.022026,0.234937,-0.260987,-0.202288,-0.390558
citric_acid,0.671703,-0.552496,1.000000,0.143577,0.203823,-0.060978,0.035533,0.364947,-0.541904,0.312770,0.109903,0.226373
residual_sugar,0.114777,0.001918,0.143577,1.000000,0.055610,0.187049,0.203028,0.355283,-0.085652,0.005527,0.042075,0.013732
chlorides,0.093705,0.061298,0.203823,0.055610,1.000000,0.005562,0.047400,0.200632,-0.265026,0.371260,-0.221141,-0.128907
free_sulfur_dioxide,-0.153794,-0.010504,-0.060978,0.187049,0.005562,1.000000,0.667666,-0.021946,0.070377,0.051658,-0.069408,-0.050656
total_sulfur_dioxide,-0.113181,0.076470,0.035533,0.203028,0.047400,0.667666,1.000000,0.071269,-0.066495,0.042947,-0.205654,-0.185100
density,0.668047,0.022026,0.364947,0.355283,0.200632,-0.021946,0.071269,1.000000,-0.341699,0.148506,-0.496180,-0.174919
pH,-0.682978,0.234937,-0.541904,-0.085652,-0.265026,0.070377,-0.066495,-0.341699,1.000000,-0.196648,0.205633,-0.057731
sulphates,0.183006,-0.260987,0.312770,0.005527,0.371260,0.051658,0.042947,0.148506,-0.196648,1.000000,0.093595,0.251397
